In [32]:
# importações
from utils import *
from models import *
from config import OUTPUT_DIR  # Importa a configuração da pasta de dados
from tensorflow.keras.models import load_model # type: ignore


# 1. Aplicabilidade do Modelo 2 no Mercado Real

In [36]:
# Lista de ações
stock_list = ['PETR4', 'VALE3', 'BBDC4','ITUB4']

# Carrega os dados de todas as ações e calcula os indicadores e salva em um dicionario
all_stock_data = load_stocks_data(stock_list)

Carregando dados para: PETR4
Carregando dados para: VALE3
Carregando dados para: BBDC4
Carregando dados para: ITUB4


In [ ]:
# Processando o Modelo 2

name_model = model_2

# Usando a Combinação 4: ("Stock Data + IT (IFR + MMS + MME)", stock_data + ifr + mms + mme),
result = process_stocks_and_save_metrics(all_stock_data, 4, name_model, save_artifacts=True) # Salva os artefatos
results_(process_stocks_and_save_metrics(all_stock_data, i + 1, name_model, save_artifacts=True))


TypeError: process_stocks_and_save_metrics() got an unexpected keyword argument 'save_artifacts'

In [2]:
# Lista de ações
stock_list = ['PETR4']

# Carrega os dados de todas as ações e calcula os indicadores e salva em um dicionario
stock_data = load_stocks_data(stock_list)

Carregando dados para: PETR4


In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import joblib

def predict_close(stock, df, model='model_2', num_combination=4, output_dir="output"):
    """
    Carrega modelos treinados, faz previsões nos dados de teste e retorna um DataFrame 
    com os valores reais e previstos desnormalizados.

    Parâmetros:
        stock (str): Nome do ativo.
        df (pd.DataFrame): DataFrame contendo os dados para previsão.
        model (str): Nome do modelo salvo. Padrão: 'model_2'.
        num_combination (int): Identificador da combinação de features usada no treinamento.
        output_dir (str): Diretório onde os modelos e scalers estão salvos.

    Retorna:
        pd.DataFrame: DataFrame contendo os dados de teste com a coluna "close_predicted" desnormalizada.
    """

    df_results = pd.DataFrame()  # DataFrame vazio para armazenar os resultados

    try:
        # 🔹 Caminhos dos arquivos salvos
        model_path = os.path.join(output_dir, f"{stock}_{model}_comb_{num_combination}.h5")
        scaler_X_path = os.path.join(output_dir, f"{stock}_{model}_comb_{num_combination}_scaler.pkl")
        scaler_y_path = os.path.join(output_dir, f"{stock}_{model}_comb_{num_combination}_scaler_y.pkl")  # Scaler para y

        # 🔹 Carregando o modelo treinado e os scalers
        model = load_model(model_path)
        scaler_X = joblib.load(scaler_X_path)
        scaler_y = joblib.load(scaler_y_path)  # Scaler para desnormalizar y

        # 🔹 Realizar a combinação de features escolhida
        df, description_combination = make_combination_data(df, num_combination)
            
        # 🔹 Definir target
        df, target_column = define_target(df)

        # 🔹 Separando variáveis independentes (X) e dependente (y)
        X = df.drop(columns=[target_column]).values
        y = df[target_column].values

        # 🔹 Dividindo os dados (80% treino, 20% teste)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        # 🔹 Normalizando os dados de teste
        X_test_scaled = scaler_X.transform(X_test)
        X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))  # Reshape para LSTM

        # 🔹 Fazendo previsões no conjunto de teste
        y_pred_scaled = model.predict(X_test_scaled).flatten()

        # 🔹 Desnormalizando as previsões
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

        # 🔹 Criando um DataFrame com os dados de teste e as previsões desnormalizadas
        df_results = df.iloc[len(X_train):].copy()  # Pegando apenas os 20% finais (testes)
        df_results["close_predicted"] = y_pred
        df_results["Stock"] = stock  # Identificador da ação

    except Exception as e:
        print(f"Erro ao processar {stock}: {e}")

    return df_results


In [17]:
for stock in stock_list:
    print(stock)

PETR4


In [26]:
for stock in stock_list:
    df_petr = predict_close(stock, stock_data[stock])

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step


In [27]:
df_petr.head()

,open,close,high,low,volume,ifr_14,mms_30,mms_5,mme_5,mme_30,close_next_day,close_predicted,Stock
date,,,,,,,,,,,,,
2019-10-23,0.502894,0.512316,0.502125,0.508677,0.065714,75.638066,0.478700,0.494929,0.498282,0.477203,0.499276,0.511140,PETR4
2019-10-24,0.515709,0.499276,0.505566,0.504077,0.086601,69.214428,0.479569,0.497661,0.498613,0.478627,0.518526,0.507621,PETR4
2019-10-25,0.516742,0.518526,0.511637,0.520385,0.106555,79.056597,0.481205,0.504492,0.505251,0.481201,0.525771,0.521176,PETR4
2019-10-28,0.517156,0.525771,0.513864,0.522685,0.053597,82.545443,0.482267,0.512068,0.512091,0.484077,0.530325,0.525661,PETR4
2019-10-29,0.520876,0.530325,0.519530,0.525612,0.071528,81.609181,0.483737,0.517243,0.518169,0.487061,0.535707,0.530115,PETR4
